# **Real-Time Gold Price Analyzer**
#### By Eshal Shanoj, 16/04/2025

*This project fetches live gold prices, forecasts future prices using ARIMA, and displays everything in an interactive Streamlit dashboard.*


**Installations**

In [19]:
!pip install yfinance keras plotly statsmodels streamlit python-dotenv pyngrok --quiet
!npm install -g localtunnel
!pip install pandas==2.2.2 --quiet
!pip install python-dotenv --quiet

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
changed 22 packages in 3s
⠸
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼

**Setup & Imports**

In [26]:
import time
import numpy as np
import pandas as pd
import streamlit as st
import yfinance as yf
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA
import plotly.graph_objects as go  # If using Plotly for charts
import dotenv
import os

dotenv.load_dotenv('/content/.env')

**Fetch Gold Price (Real-time with Fallback)**

In [21]:
USE_SIMULATED_DATA = False  # Set to True for testing with simulated data

def fetch_yahoo_finance():
    try:
        data = yf.download("GC=F", period="1d", interval="1m")
        if not data.empty:
            return data['Close'].iloc[-1]
    except Exception as e:
        print(f"Yahoo Finance error: {e}")
    return None

@st.cache_data(ttl=60)  # Cache for 1 minute
def fetch_gold_price():
    if USE_SIMULATED_DATA:
        return 3260.0 + np.random.normal(0, 1)  # Updated baseline

    # Try Yahoo Finance first
    try:
        price = fetch_yahoo_finance()
        if price and 2500 < price < 4000:  # Reasonable range check
            return price
    except Exception as e:
        print(f"Yahoo Finance primary error: {e}")

    # If Yahoo Finance fails, return default
    return 3260.0  # Updated fallback based on latest data

2025-04-17 07:24:48.669 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


**Real-Time Streamer**

In [22]:
# Updated chart creation code
def create_price_chart(actual_df, forecast_values, forecast_dates):
    fig = go.Figure()

    # Actual prices
    fig.add_trace(go.Scatter(
        x=actual_df['Time'],
        y=actual_df['Price'],
        name='Actual',
        line=dict(color='blue'),
        mode='lines+markers'
    ))

    # Forecast (only if data exists)
    if forecast_values and forecast_dates and len(forecast_values) == len(forecast_dates):
        fig.add_trace(go.Scatter(
            x=forecast_dates,
            y=forecast_values,
            name='Forecast',
            line=dict(color='red', dash='dash'),
            mode='lines+markers'
        ))

    fig.update_layout(
        title='Gold Price: Actual vs Forecast',
        xaxis_title='Time',
        yaxis_title='Price (USD)',
        xaxis=dict(tickformat='%H:%M'),
        hovermode='x unified'
    )
    return fig

    fig.update_layout(
        title='Gold Price: Actual vs Forecast',
        xaxis_title='Date & Time',
        yaxis_title='Price (USD)',
        hovermode='x unified',
        xaxis=dict(
            tickformat='%b %d %H:%M',
            rangeslider=dict(visible=True))
        )
    return fig

Streamlit App

In [23]:
%%writefile app.py
# app.py
import streamlit as st
import time
import numpy as np
import pandas as pd
import os
import requests
from datetime import datetime, timedelta
import yfinance as yf
from statsmodels.tsa.arima.model import ARIMA
import plotly.graph_objects as go

# Configuration
st.set_page_config(page_title="Gold Price Analysis & Forecasting", layout="wide")
st.title("💰 Real-Time Gold Price Analyzer")

# Constants
USE_SIMULATED_DATA = False
SIMULATED_BASELINE = 3260
REFRESH_INTERVAL = 60
FORECAST_HORIZON = 12
FORECAST_FREQ = '5min'

# Initialize session state
if 'prices' not in st.session_state:
    st.session_state.prices = []
if 'timestamps' not in st.session_state:
    st.session_state.timestamps = []
if 'forecasts' not in st.session_state:
    st.session_state.forecasts = []
if 'forecast_dates' not in st.session_state:
    st.session_state.forecast_dates = []
if 'last_price' not in st.session_state:
    st.session_state.last_price = 0.0
if 'refresh_counter' not in st.session_state:
    st.session_state.refresh_counter = 0

@st.cache_data(ttl=300)
def fetch_gold_price():
    if USE_SIMULATED_DATA:
        return SIMULATED_BASELINE + np.random.normal(0, 50)  # <<< Updated volatility

    price = fetch_yahoo_spot()
    if price is not None and 2500 < price < 4000:
        return price

    return 3260.0  # Fallback to current market price

def fetch_yahoo_spot():
    data = yf.download("GC=F", period="1d", interval="1m", progress=False)
    if not data.empty:
        return float(data['Close'].iloc[-1])
    return None

@st.cache_data(ttl=3600)
def get_historical_data():
    if USE_SIMULATED_DATA:
        dates = pd.date_range(end=datetime.now(), periods=180)
        prices = 1800 + np.cumsum(np.random.normal(0, 10, 180))
        return pd.Series(prices, index=dates)

    try:
        return yf.download("GC=F", period="6mo", interval="1d")['Close']
    except:
        dates = pd.date_range(end=datetime.now(), periods=180)
        prices = 1800 + np.cumsum(np.random.normal(0, 10, 180))
        return pd.Series(prices, index=dates)

# --- Forecasting Functions ---
def forecast_next_price(prices, steps=12, freq='5min'):
    try:
        # Convert to pandas Series if it's a list
        if isinstance(prices, list):
            series = pd.Series(prices)
        else:
            series = prices

        model = ARIMA(series, order=(5,1,0))
        model_fit = model.fit()
        forecast = model_fit.forecast(steps=steps)

        # Generate future dates starting from now
        forecast_dates = [datetime.now() + timedelta(minutes=5*i) for i in range(1, steps+1)]

        # Return forecast values and dates
        return forecast.tolist(), forecast_dates
    except Exception as e:
        st.sidebar.error(f"Forecasting error: {e}")
        last_price = prices[-1] if isinstance(prices, list) else prices.iloc[-1]

        # Simple fallback forecast (slight increase)
        forecast_values = [last_price * (1 + 0.0001*i) for i in range(1, steps+1)]
        forecast_dates = [datetime.now() + timedelta(minutes=5*i) for i in range(1, steps+1)]

        return forecast_values, forecast_dates

# --- Visualization Functions ---
def create_price_chart(actual_df, forecast_values, forecast_dates):
    fig = go.Figure()

    # Actual prices
    fig.add_trace(go.Scatter(
        x=actual_df['Time'],
        y=actual_df['Price'],
        name='Actual',
        line=dict(color='blue'),
        mode='lines+markers'
    ))

    # Forecast
    if len(forecast_values) > 0 and len(forecast_dates) > 0:
        fig.add_trace(go.Scatter(
            x=forecast_dates,
            y=forecast_values,
            name='Forecast',
            line=dict(color='red', dash='dash'),
            mode='lines+markers'
        ))

    fig.update_layout(
        title='Gold Price: Actual vs Forecast',
        xaxis_title='Time',
        yaxis_title='Price (USD)',
        xaxis=dict(tickformat='%H:%M'),
        hovermode='x unified'
    )
    return fig

# --- Performance Metrics Functions ---
def calculate_metrics(prices, forecasts):
    if len(prices) < 2 or len(forecasts) == 0:
        return {}

    # We can only evaluate forecasts that have corresponding actual values
    # So we use the forecast from time t-1 and compare to actual at time t
    if len(prices) > len(forecasts):
        actuals = prices[-len(forecasts):]
    else:
        actuals = prices[1:]  # Skip first actual
        forecasts = forecasts[:len(actuals)]  # Truncate forecasts

    errors = [a - f for a, f in zip(actuals, forecasts)]

    return {
        'MAE': np.mean(np.abs(errors)) if errors else 0,
        'RMSE': np.sqrt(np.mean(np.square(errors))) if errors else 0,
        'MAPE': np.mean(np.abs([e/a*100 for e, a in zip(errors, actuals) if a != 0])) if errors else 0,
        'Last_Error': errors[-1] if errors else None,
        'Volatility': np.std(prices[-30:]) if len(prices) >= 30 else np.std(prices)
    }

# --- Streamlit Dashboard Layout ---
col1, col2 = st.columns([3, 1])

with col2:
    st.subheader("Live Gold Price")
    price_display = st.empty()
    change_display = st.empty()

    st.subheader("Forecast")
    forecast_display = st.empty()

    # User input for forecast horizon
    forecast_minutes = st.slider(
        "Forecast minutes ahead",
        min_value=5,
        max_value=60,
        value=5,
        step=5
    )

    st.subheader("Analysis")
    metrics_container = st.container()

    with st.expander("Debug Info"):
        st.write(f"Refresh count: {st.session_state.refresh_counter}")
        st.write(f"Data points: {len(st.session_state.prices)}")

with col1:
    chart = st.empty()

# --- Main App Logic ---
# Main Data Processing
current_price = fetch_gold_price()
historical = get_historical_data()

# Handle None price
if current_price is None:
    current_price = 1800.0

# Calculate price change
price_change = 0
if st.session_state.last_price > 0:
    price_change = current_price - st.session_state.last_price

# Update session state
st.session_state.timestamps.append(datetime.now())
st.session_state.prices.append(current_price)
st.session_state.last_price = current_price
st.session_state.refresh_counter += 1

# Generate forecasts
if len(st.session_state.prices) >= 5:
    forecast_values, forecast_dates = forecast_next_price(
        st.session_state.prices,
        steps=FORECAST_HORIZON,
        freq=FORECAST_FREQ
    )
    st.session_state.forecasts = forecast_values
    st.session_state.forecast_dates = forecast_dates

# Prepare data for display
actual_df = pd.DataFrame({
    'Time': st.session_state.timestamps,
    'Price': st.session_state.prices
})

# Keep only recent data
if len(actual_df) > 60:
    actual_df = actual_df.iloc[-60:]

# Display chart
fig = create_price_chart(actual_df, st.session_state.forecasts, st.session_state.forecast_dates)
chart.plotly_chart(fig, use_container_width=True)

# Update metrics displays
price_display.metric(
    "Current Price",
    f"${current_price:.2f}" if current_price is not None else "Data Unavailable",
    f"{price_change:.2f}" if price_change != 0 and price_change is not None else None
)

change_display.markdown(
    "⬆️ Rising" if price_change > 0 else
    "⬇️ Falling" if price_change < 0 else
    "➡️ Stable"
)

# Show forecast for selected horizon
if len(st.session_state.forecasts) > 0:
    steps_ahead = forecast_minutes // 5
    if steps_ahead <= len(st.session_state.forecasts):
        selected_forecast = st.session_state.forecasts[steps_ahead - 1]
        forecast_change = selected_forecast - current_price
        forecast_display.metric(
            f"Next {forecast_minutes}min Forecast",
            f"${selected_forecast:.2f}",
            f"{forecast_change:.2f}"
        )
    else:
        forecast_display.info(f"Forecast for {forecast_minutes}min not available. Max: {len(st.session_state.forecasts) * 5}min")

# Calculate and display performance metrics
metrics = calculate_metrics(st.session_state.prices, st.session_state.forecasts)
with metrics_container:
    if metrics:
        st.metric("Volatility (σ)", f"{metrics['Volatility']:.4f}")
        cols = st.columns(2)
        cols[0].metric("MAE", f"{metrics['MAE']:.4f}")
        cols[1].metric("RMSE", f"{metrics['RMSE']:.4f}")
        if 'MAPE' in metrics:
            cols[0].metric("MAPE (%)", f"{metrics['MAPE']:.2f}")
        if metrics['Last_Error'] is not None:
            cols[1].metric("Last Error", f"{metrics['Last_Error']:.4f}")

# Auto-refresh
time.sleep(REFRESH_INTERVAL)
st.rerun()  # Updated from experimental_rerun


Overwriting app.py


**Launch Everything** (Colab compatible)

In [24]:
# Import
from pyngrok import ngrok
import threading
import os

# Set auth token
import os
ngrok.set_auth_token(os.environ["NGROK_AUTHTOKEN"])

try:
    # Check for existing tunnels
    tunnels = ngrok.get_tunnels()
    print(f"Active tunnels: {len(tunnels)}")

    # Close existing tunnels
    for tunnel in tunnels:
        ngrok.disconnect(tunnel.public_url)
        print(f"Closed: {tunnel.public_url}")
except Exception as e:
    print(f"Error checking tunnels: {e}")
    # Continue anyway - this is just cleanup

# Start a new tunnel
try:
    public_url = ngrok.connect("8501", "http")
    print(f"📊 Streamlit Dashboard: {public_url}")
except Exception as e:
    print(f"Error starting ngrok: {e}")
    print("Trying to run Streamlit without ngrok...")

# Start Streamlit in a separate thread
def start_streamlit():
    os.system("streamlit run app.py --server.port 8501")

threading.Thread(target=start_streamlit).start()

Active tunnels: 1
Closed: https://962f-34-55-120-169.ngrok-free.app
📊 Streamlit Dashboard: NgrokTunnel: "https://b1ca-34-55-120-169.ngrok-free.app" -> "http://localhost:8501"


In [25]:
import yfinance as yf
data = yf.download("GC=F", period="1d", interval="1m")
print(data.tail())

[*********************100%***********************]  1 of 1 completed

Price                            Close         High          Low         Open  \
Ticker                            GC=F         GC=F         GC=F         GC=F   
Datetime                                                                        
2025-04-17 07:10:00+00:00  3336.500000  3338.000000  3336.300049  3337.600098   
2025-04-17 07:11:00+00:00  3336.600098  3337.000000  3336.300049  3336.399902   
2025-04-17 07:12:00+00:00  3336.000000  3336.800049  3335.100098  3336.399902   
2025-04-17 07:13:00+00:00  3334.300049  3335.899902  3334.100098  3335.800049   
2025-04-17 07:14:00+00:00  3335.399902  3335.600098  3333.100098  3334.199951   

Price                     Volume  
Ticker                      GC=F  
Datetime                          
2025-04-17 07:10:00+00:00    150  
2025-04-17 07:11:00+00:00    136  
2025-04-17 07:12:00+00:00    260  
2025-04-17 07:13:00+00:00    304  
2025-04-17 07:14:00+00:00      0  
